# Introudction to xDSL / MLIR concepts

xDSL is a framework to make it easy to define programming languages. These languages can provide abstractions levels which, for example, enables compiler to perform optimisations more effectively, by making information more accessible to the compiler.

## Static Single Assignment (SSA) 
The kind of programming languages xDSL defines all have the $\textit{static single assignment}$ (SSA) property. It is a property on variables that means they can be assigned a value only once, and once assigned a value, they cannot be modified. This is a common restriction used within compilers, as it enables many compiler optimisations to be performed.

## xDSL
A major focus of xDSL (and MLIR) is on reusability. 

Therefore, often fragments of a language is defined, rather than a full language, which allows them to be used in many situations. Some examples include:
* `arith`, which contains arithmetic operations such as `add`, `multiply`, and types such as `float`s, `int`s.
* `scf`, which contains structural control flow operations such as `if-else`, and `while` loops.

These fragments of languages are referred to as a `dialect`. A dialect is comprised of:
* a collection of attributes. attributes represent information which are available during compile time. Attributes can be:
    * types - (for instance `float` within `arith` dialect; `memref` within `memref` dialect, representing pointers / references)
    * data - (these are extra compile-time information provided within the IR, 
* a collection of operations (

As such, languages defined in xDSL can be combined together, where the combined language can benefit from the optimisations of all its constituents.

When a DSL is defined in xDSL, it comes 